<a href="https://colab.research.google.com/github/XxBlessingx/Natural-Language-Processing/blob/main/Fake_News_Classification_lab_and_ACW_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install transformers datasets torch

In [ ]:
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
'/content/drive/MyDrive/fakenews/Fake.csv'

'/content/drive/MyDrive/fakenews/Fake.csv'

In [ ]:
'/content/drive/MyDrive/fakenews/True.csv'

'/content/drive/MyDrive/fakenews/True.csv'

In [ ]:
!ls /content/drive/MyDrive/Colab\ Notebooks/fakenews/

ls: cannot access '/content/drive/MyDrive/Colab Notebooks/fakenews/': No such file or directory


In [ ]:
!pip install Dataset

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 231.9/231.9 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 57.6 MB/s eta 0:00:00
  Attempting uninstall: sqlalchemy
    Found existing installation: SQLAlchemy 2.0.40
    Uninstalling SQLAlchemy-2.0.40:
      Successfully uninstalled SQLAlchemy-2.0.40
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython-sql 0.5.0 requires sqlalchemy>=2.0, but you have sqlalchemy 1.4.54 which is incompatible.


Step 1: Loading Libraries  and printing the dataset

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer,Trainer,TrainingArguments, BertForSequenceClassification
from datasets import Dataset, DatasetDict

# Load the real and fake news datasets with correct paths
real_news = pd.read_csv('/content/drive/MyDrive/fakenews/True.csv')  # File for true news
fake_news = pd.read_csv('/content/drive/MyDrive/fakenews/Fake.csv')  # File for fake news

# Add labels for classification
real_news['label'] = 0  # Label for real news
fake_news['label'] = 1  # Label for fake news

# Combine the datasets
data = pd.concat([real_news, fake_news], ignore_index=True)

data = data[["title", "text", "label"]]

train_texts, test_texts, train_labels, test_labels = train_test_split(
    data["text"].tolist(),
    data["label"].tolist(),
    test_size=0.2, random_state=42
)
debug = True
if debug:
  train_texts = train_texts[:5000]
  train_labels = train_labels[:5000]
  test_texts = test_texts[:1000]
  test_labels = test_labels[:1000]



Step 2: Tokenization

In [ ]:
from datasets import Dataset, DatasetDict
from transformers import BertTokenizer

# Splitting the dataset into training and testing
train_dataset = Dataset.from_dict({"text": train_texts, "label": train_labels})
test_dataset = Dataset.from_dict({"text": test_texts, "label": test_labels})

# Create a DatasetDict
dataset = DatasetDict({"train": train_dataset, "test": test_dataset})

# Load the tokenizer
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

# Tokenization function
def tokenize_function(examples):
    return tokenizer(examples["text"], padding=True, truncation=True)

# Apply tokenization to the datasets
train_dataset = train_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)

# Inspect the first item to confirm `input_ids` are present
print(train_dataset[0])  # Check if 'input_ids' are correctly tokenized


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

{'text': 'This story is from 2006 and is the first in a long series of Hillary s lies that we ll be exposing over the next couple of months For more than a decade, one piece of Senator Hillary Rodham Clinton s informal biography has been that she was named for Sir Edmund Hillary, the conqueror of Mount Everest. The story was even recounted in Bill Clinton s autobiography.But yesterday, Mrs. Clinton s campaign said she was not named for Sir Edmund after all. It was a sweet family story her mother shared to inspire greatness in her daughter, to great results I might add,  said Jennifer Hanley, a spokeswoman for the campaign.In May 1953, Sir Edmund and his Sherpa guide, Tenzing Norgay, became the first men to reach the summit of Mount Everest. In 1995, shortly after meeting Sir Edmund, Mrs. Clinton said that her mother, Dorothy Rodham, had long told her she was named for the famous mountaineer.  It had two l s, which is how she thought she was supposed to spell Hillary,  Mrs. Clinton said

Step 3: Model setup and training

In [ ]:
from transformers import BertForSequenceClassification, Trainer, TrainingArguments

model_name = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)

# set up training arguements
training_args = TrainingArguments(
    output_dir="./results",          # where the model is saved
    num_train_epochs=2,              # how many times the model goes through the data
    per_device_train_batch_size=8,  # breaks batchs into smaller sizes for training
    gradient_accumulation_steps=4,   # Accumulate gradients over 2 steps (effectively simulating a batch size of 48)
    per_device_eval_batch_size=8,   # how many samples to test at once during evaluation
    warmup_steps=500,                # number of steps to slowly increase learning rate
    weight_decay=0.01,               # helps the model from overfitting
    logging_dir="./logs",            # where the training log is saved
    logging_steps=10,                # how often the training log progress

)
# intialize Trainer
trainer = Trainer(
    model=model_name,                    # the model to be trained
    args=training_args,                  # the training arguments
    train_dataset=train_dataset,         # the training dataset
    eval_dataset=test_dataset,           # the evaluation dataset
)

# traing the model
trainer.train()

# Evaluate the model
results = trainer.evaluate()
print(results)

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: b-coxe-2022 (b-coxe-2022-university-of-hull) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss
10,0.707600
20,0.683300
30,0.662600
40,0.591000
50,0.523500
60,0.448000
70,0.344300
80,0.274000
90,0.193700
100,0.131500


{'eval_loss': 0.007517303805798292, 'eval_runtime': 30.3211, 'eval_samples_per_second': 32.98, 'eval_steps_per_second': 4.123, 'epoch': 1.992}


Printing out the key information needed for the tables

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Get predictions from the model
predictions = trainer.predict(test_dataset)

# Convert model outputs (logits) into predicted class labels
pred_labels = predictions.predictions.argmax(-1)

# Extract the true labels
true_labels = predictions.label_ids

# Calculate and print evaluation metrics
accuracy = accuracy_score(true_labels, pred_labels)
precision = precision_score(true_labels, pred_labels, average='binary')
recall = recall_score(true_labels, pred_labels, average='binary')
f1 = f1_score(true_labels, pred_labels, average='binary')

print(f"Accuracy:  {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall:    {recall:.4f}")
print(f"F1-score:  {f1:.4f}")


Accuracy: 0.999
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       464
           1       1.00      1.00      1.00       536

    accuracy                           1.00      1000
   macro avg       1.00      1.00      1.00      1000
weighted avg       1.00      1.00      1.00      1000



Step 4: Expriementing with different forms of input when feeding into  BERT model Title-Only vs. Title and Text

In [ ]:
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")


def tokenize_function(example):
    return tokenizer(example["text"], padding="max_length", truncation=True, max_length=512)


Training with the full text dataset

In [ ]:
# Reduce full-text input and labels together
train_texts_full = data['text'].tolist()[:5000]
test_texts_full = data['text'].tolist()[:1000]

train_labels = data['label'].tolist()[:5000]
test_labels = data['label'].tolist()[:1000]

# Now create datasets
train_dataset_full = Dataset.from_dict({"text": train_texts_full, "label": train_labels})
test_dataset_full = Dataset.from_dict({"text": test_texts_full, "label": test_labels})


# Tokenize full-text dataset
train_dataset_full = train_dataset_full.map(tokenize_function, batched=True)
test_dataset_full = test_dataset_full.map(tokenize_function, batched=True)

# Train and evaluate the full-text model
trainer_full_text = Trainer(
    model=model_name,
    args=training_args,
    train_dataset=train_dataset_full,
    eval_dataset=test_dataset_full,
)

trainer_full_text.train()
results_full_text = trainer_full_text.evaluate()

print("Full-text Model Results:", results_full_text)

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Step,Training Loss
10,0.000400
20,0.000400
30,0.000300
40,0.000300
50,0.000500
60,0.000200
70,0.000200
80,0.000100
90,0.000100
100,0.000100


Full-text Model Results: {'eval_loss': 5.981666163279442e-06, 'eval_runtime': 30.3301, 'eval_samples_per_second': 32.971, 'eval_steps_per_second': 4.121, 'epoch': 1.992}


Training with the title-only dataset

In [ ]:
# Title-only dataset
train_texts_title_only = data['title'].tolist()[:5000]
test_texts_title_only = data['title'].tolist()[:1000]

train_labels = data['label'].tolist()[:5000]
test_labels = data['label'].tolist()[:1000]

train_dataset_title_only = Dataset.from_dict({"text": train_texts_title_only, "label": train_labels})
test_dataset_title_only = Dataset.from_dict({"text": test_texts_title_only, "label": test_labels})

# Tokenize title-only dataset
train_dataset_title_only = train_dataset_title_only.map(tokenize_function, batched=True)
test_dataset_title_only = test_dataset_title_only.map(tokenize_function, batched=True)

# Train and evaluate the title-only model (using the same setup as for full text)
trainer_title_only = Trainer(
    model=model_name,
    args=training_args,
    train_dataset=train_dataset_title_only,
    eval_dataset=test_dataset_title_only,
)

trainer_title_only.train()
results_title_only = trainer_title_only.evaluate()

print("Title-only Model Results:", results_title_only)


Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Step,Training Loss
10,0.000600
20,0.000000
30,0.000000
40,0.000000
50,0.000100
60,0.000500
70,0.000000
80,0.000000
90,0.000000
100,0.000000


Title-only Model Results: {'eval_loss': 1.8682461586649879e-06, 'eval_runtime': 30.2287, 'eval_samples_per_second': 33.081, 'eval_steps_per_second': 4.135, 'epoch': 1.992}


Step 5: Comparing other models: DistilBERT

---



Gathering the dataset and assigning labels from the dataset to test and train

In [ ]:
from datasets import Dataset

train_dataset_distil = Dataset.from_dict({
    "text": train_texts,
    "label": train_labels
})

test_dataset_distil = Dataset.from_dict({
    "text": test_texts,
    "label": test_labels
})


tokenizening the training and testing data

In [ ]:
from transformers import DistilBertTokenizer

# Initialize tokenizer
distil_tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")

# Define tokenizer function
def tokenize_distilbert(example):
    return distil_tokenizer(example["text"], padding="max_length", truncation=True, max_length=512)

# Apply tokenizer to dataset
train_dataset_distil = train_dataset_distil.map(tokenize_distilbert, batched=True)
test_dataset_distil = test_dataset_distil.map(tokenize_distilbert, batched=True)


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [ ]:
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification

distil_model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased")
distil_tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")

from transformers import TrainingArguments, Trainer

training_args_distil = TrainingArguments(
    output_dir="./results_distilbert",
    num_train_epochs=2,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    warmup_steps=100,
    weight_decay=0.01,
    logging_dir='./logs_distilbert',
    logging_steps=10
    #evaluation_strategy="epoch"
)

trainer_distil = Trainer(
    model=distil_model,
    args=training_args_distil,
    train_dataset=train_dataset_distil,
    eval_dataset=test_dataset_distil
)
# Train the DistilBERT model
trainer_distil.train()

# Evaluate on the test set
results_distil = trainer_distil.evaluate()

# printing the results
print(results_distil)


Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Step,Training Loss
10,0.569500
20,0.419900
30,0.176500
40,0.042200
50,0.010700
60,0.004300
70,0.002400
80,0.001500
90,0.001000
100,0.000800


{'eval_loss': 1.01709269983985e-06, 'eval_runtime': 15.2141, 'eval_samples_per_second': 65.729, 'eval_steps_per_second': 8.216, 'epoch': 2.0}


In [ ]:
from sklearn.metrics import classification_report

predictions = trainer_distil.predict(test_dataset_distil)
pred_labels = predictions.predictions.argmax(-1)

print(classification_report(test_labels, pred_labels))


              precision    recall  f1-score   support

           0       1.00      1.00      1.00      1000

    accuracy                           1.00      1000
   macro avg       1.00      1.00      1.00      1000
weighted avg       1.00      1.00      1.00      1000

